In [1]:
### If you haven't yet installed wot, uncomment and run the line below:
#!pip install -q wot

### We begin by importing some useful python packages:
import wot
import matplotlib.pyplot as plt

# Notebook 3: Computing long-range couplings

In Notebook 2 we saw how to compute transport matrices $\pi_{t_i,t_{i+1}}$ connecting adjacent time-points $(t_i,t_{i+1})$. Over short time-scales, this OT matrix is a good approximation to the true temporal coupling:

$$ \pi_{t_i,t_{i+1}} \approx \gamma_{t_i,t_{i+1}}. $$

In this notebook we show how to infer transitions over a longer time interval $(t_i,t_j)$.
To do this, we assume the developmental stochastic process ${\mathbb{P}}_t$ is Markov. This means that the couplings $\gamma_{s,t}$ satisfy the following algebraic relationship:

$$ \gamma_{t_1,t_3}(x,z) = \iint \gamma_{t_1,t_2}(x,y) \gamma_{t_2,t_3}(y,z) dy$$

for any $t_1 < t_2 < t_3$. The integral on the right hand side becomes a finite sum when we have a finite sample of cells. In fact, it's a matrix multiplication! 

Therefore we can infer long-range transitions by composing transport maps as follows:

$$\gamma_{t_i,t_j} = \gamma_{t_i,t_{i+1}} \circ \gamma_{t_{i+1},t_{i+2}} \circ \cdots \circ \gamma_{t_{j-1},t_{j}} 
\approx \pi_{t_i,t_{i+1}} \circ \pi_{t_{i+1},t_{i+2}} \circ \cdots \circ \pi_{t_{j-1},t_{j}}.$$

Here $\circ$ denotes matrix multiplication. 

In the following code block we construct a TransportMapModel from a directory of pre-computed transport matrices. 

In [3]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')

# Composing copulings

We can now easily compute the coupling between any pair of time-points as follows:

In [4]:
gamma_8_10 = tmap_model.get_coupling(10, 8)
gamma_8_10

AnnData object with n_obs × n_vars = 6507 × 3815 
    obs: 'g0', 'g1', 'g2', 'g3'

The resulting temporal coupling $\gamma_{t_i,t_j}$ has a row for each cell at time $t_i$ and a column for each cell at time $t_j$. 
Just like for a short-range coupling, the units are "transported mass". So a value of $\gamma_{t_i,t_j}(x,y) = 0.2$ means that cell $x$ will have on average $0.2$ descendants with expression profile similar to $y$ at time $t_j$.
Note that the sum of a row shows the total number of descendants that a cell will have at time $t_j$.

# Composing couplings vs direct transport

We Composing the coupling 

In [ ]:
VAR_GENE_DS_PATH = 'data/ExprMatrix.var.genes.loom'
CELL_DAYS_PATH = 'data/cell_days.txt'
SERUM_CELL_IDS_PATH = 'data/serum_cell_ids.txt'
CELL_GROWTH_PATH = 'data/growth_gs_init.txt'

### load data
adata = wot.io.read_dataset(VAR_GENE_DS_PATH, obs=[CELL_DAYS_PATH, CELL_GROWTH_PATH], obs_filter=SERUM_CELL_IDS_PATH)
ot_model = wot.ot.OTModel(adata,epsilon = 0.05, lambda1 = 1,lambda2 = 50) 
tmap_7_8 = 